In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model

from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
#from keras.utils import to_categorical1
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/User history1.csv',delimiter=',',encoding='latin-1')
df.head()

,Username,Summary,Userhistory,label
0,Kini_Monei,sad,I'm so sadThe one good thing that's gonna come...,1
1,MartensCentre,sad,It was a pleasure to have you again with us in...,0
2,jjkraps,sad,im so sorry im looking at old mentions byebrea...,1
3,DaroganY,sad,I fear for Wales. When everything is divided u...,1
4,medwards2111,sad,Stay Encouraged - For God has not given us the...,0


In [45]:
df.drop(['Username','Summary'],axis=1,inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1212 entries, 0 to 1211
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Userhistory  1212 non-null   object
 1   label        1212 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 19.1+ KB


In [46]:
X = df.Userhistory 
Y = df.label
le = LabelEncoder()
Y = le.fit_transform(Y)
#le.classes_=['no','yes']
print(le.classes_)
Y = Y.reshape(-1,1)


[0 1]


In [47]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)
print(Y_train)


[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]


In [48]:
max_words = 1000
max_len = 180
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [49]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [50]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 180)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 180, 50)           50000     
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                

In [51]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=3,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/3
6/6 [==============================] - 2s 250ms/step - loss: 0.6633 - accuracy: 0.6873 - val_loss: 0.6165 - val_accuracy: 0.7059
Epoch 2/3
6/6 [==============================] - 1s 171ms/step - loss: 0.5845 - accuracy: 0.7316 - val_loss: 0.5942 - val_accuracy: 0.7059
Epoch 3/3
6/6 [==============================] - 1s 174ms/step - loss: 0.5707 - accuracy: 0.7316 - val_loss: 0.6150 - val_accuracy: 0.7059


In [52]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

12/12 [==============================] - 0s 20ms/step - loss: 0.5922 - accuracy: 0.7088


In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.592
  Accuracy: 0.709


In [53]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [54]:
SINGLE_ATTENTION_VECTOR = False
APPLY_ATTENTION_BEFORE_LSTM = False
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    # output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul



In [60]:
def model_attention_applied_after_lstm():
    #inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    
    lstm_units = 128
    lstm_out = LSTM(lstm_units, return_sequences=True)(layer)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
    model = Model(inputs=[inputs], outputs=output)
    return model

In [62]:
from keras.layers import merge
from keras.layers import multiply
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *

from keras.utils.vis_utils import plot_model

In [63]:
INPUT_DIM = 50
TIME_STEPS = max_len

In [64]:
sequences_matrix.shape,Y_train.shape

((848, 180), (848, 1))

In [65]:
m = model_attention_applied_after_lstm()

In [66]:
m.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 180)]        0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 180, 50)      50000       inputs[0][0]                     
__________________________________________________________________________________________________
lstm_6 (LSTM)                   (None, 180, 128)     91648       embedding_6[0][0]                
__________________________________________________________________________________________________
permute_4 (Permute)             (None, 128, 180)     0           lstm_6[0][0]                     
_______________________________________________________________________________________

In [67]:
#model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer='adam')


m.compile(optimizer=RMSprop(lr=0.1), loss='binary_crossentropy', metrics=['accuracy'])
m.fit(sequences_matrix,Y_train,batch_size=256,epochs=3,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/3
3/3 [==============================] - 3s 1s/step - loss: 6.3145 - accuracy: 0.6195 - val_loss: 3.8542 - val_accuracy: 0.3118
Epoch 2/3
3/3 [==============================] - 3s 876ms/step - loss: 3.6716 - accuracy: 0.4705 - val_loss: 3.6276 - val_accuracy: 0.7059
Epoch 3/3
3/3 [==============================] - 3s 870ms/step - loss: 2.0561 - accuracy: 0.6696 - val_loss: 1.5283 - val_accuracy: 0.7000


In [68]:
accr = m.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.4f}\n  Accuracy: {:0.4f}'.format(accr[0],accr[1]))

12/12 [==============================] - 1s 86ms/step - loss: 1.3287 - accuracy: 0.7170
Test set
  Loss: 1.3287
  Accuracy: 0.7170
